In [4]:
import pandas as pd
import re
from geopy.geocoders import Nominatim

In [5]:
loc = pd.read_csv('all_locations.csv')
df = pd.read_csv('Master_checklist.csv')

In [6]:
loc['Locations'] = loc['Locations'].str.strip()
df.drop(columns='Unnamed: 0', inplace=True)

The All Locations table holds all Standard Naming Conventions of existing cities found in the Master Checklist. If there is a city not yet found in this table. The script will geo-locate the city and add it as a new record. This table also holds correct coordinate information for each city, which is used to cross-reference and correct the Master Checklist where necessary. 

In [7]:
loc

,Locations,Lat,Long
0,"Denver, CO",39.73,-104.99
1,"Newark, NJ",40.14,-74.67
2,Washington DC,38.89,-77.03
3,"Seattle, WA",47.60,-122.32
4,"San Francisco, CA",37.73,-122.46
5,"New York City, NY",42.93,-75.61
6,"Santa Fe, NM",35.69,-105.93
7,"Dallas, TX",32.77,-96.79
8,"Jackson, WY",43.47,-110.76
9,"Portland, OR",45.51,-122.67


The Master Checklist is the working database holding individuals location data from where the Interview was conducted, where they came from (i.e. Migrated From), where the Migrate To, and where they are currently calling home. 

In [8]:
df

,Interview,Interview lat,Interview long,Migrated From,Migrated From lat,Migrated From long,Migrated To,Migrated To lat,Migrated To long,Current Residence,Current Residence lat,Current Residence long
0,"Denver, CO",39.73,-104.99,Los Angeles,34.0500,-118.2400,"Dallas, TX",32.78,-96.79,"Dallas, TX",32.7800,-96.7900
1,"Newark, NJ",40.14,-74.67,"Dallas, TX",32.7700,-96.7900,"Jackson, WY",43.47,-110.76,"Jackson, WY",43.4700,-110.7600
2,Washington DC,40.22,-77.03,Washington DC,34.5623,12.7685,"Portland, OR",45.51,-122.67,"Portland, OR",45.5100,-122.6700
3,"Seattle, WA",47.60,-122.32,"Seattle, WA",47.6000,-122.3200,Los Angeles,34.05,-118.24,Los Angeles,34.0500,-118.2400
4,"San Francisco, CA",37.73,-122.46,Washington DC,23.4543,45.6834,"Newark, NJ",40.14,-74.67,"Newark, NJ",40.1400,-74.6700
5,"New York City, NY",42.93,-75.61,"Newark, NJ",40.1400,-74.6700,"Denver, CO",39.73,-104.99,"Denver, CO",39.7300,-104.9900
6,"Santa Fe, NM",35.69,-105.93,"Denver, CO",39.7300,-104.9900,"Denver, CO",39.73,-104.99,"Denver, CO",39.7300,-120.9900
7,"Dallas, TX",32.77,-96.79,"Jackson, WY",43.4700,-110.7600,"Denver, CO",39.73,-104.99,"Denver, CO",39.7300,-104.9900
8,"Jackson, WY",43.47,-110.76,"Portland, OR",45.5100,-122.6700,"Denver, CO",39.73,-103.99,"Denver, CO",39.7300,-104.9900
9,"Portland, OR",45.51,-122.67,"New York City, NY",41.9300,-75.6100,"San Francisco, CA",37.73,-122.46,"San Francisco, CA",39.5555,34.2345


The function below updates both csvs, if there are discrepancies. If a location is not yet in the All Locations list it will be added. If the coordinates in the master checklist are incorrect they will be updated based on the existing coordinates found in the All Locations list. The function takes a column name as a parameter to filter the dataframe and only work with the field and its corresponding lat/long.  A regular expression is forced to control the event when there is a naming convention slightly different to what is stated in the All Locations. 

For example, if 'Los Angeles, CA' is in the All locations but 'Los Angeles' is in the Master Checklist. The regular expression will catch this naming discrepancy and update the Master Checklist to match the All Locations. It will NOT add it as a new city in the All Locations.

In [9]:
# function that joins by either Interview, MigratedFrom, MigratedTo, CurrentResidence 
# and assess locational accuracy
def update_locations(df, col, locs):
    df_copy = df.copy()
    df_copy = df_copy[[col, f'{col} lat', f'{col} long']]
    
    # new location found in Master List
    join = df_copy.merge(locs, left_on=col, right_on='Locations', how='left')
    if join.loc[join['Locations'].isnull()].shape[0] > 0:
        for i in join.loc[join['Locations'].isnull(), col]:
            geoloc = Nominatim(user_agent="a")
            location = geoloc.geocode(i)
            
            # regex to match string if locaiton exisit but naming is different
            regex = locs.loc[locs['Locations'].str.contains(fr'\b{i}\b', case=False), 'Locations']
            if regex.shape[0] > 0:
                re = regex.to_list()[0]
                join.loc[join[col] == i, col] = re
                join.loc[join[col] == i, f'{col} lat'] = locs.loc[locs['Locations'] == re, 'Lat']
                join.loc[join[col] == i, f'{col} long'] = locs.loc[locs['Locations'] == re, 'Long']
        
            else:
                locs.loc[len(locs.index)] = [i, location.latitude, location.longitude]
                join.loc[join[col] == i, f'{col} lat'] = location.latitude
                join.loc[join[col] == i, f'{col} long'] = location.longitude

    # update location names
    join.loc[join['Locations'].notnull(), col] = join['Locations']
    
    # update coords which are incorrect
    join.loc[(join['Locations'].notnull()) & (join[col].notnull()) &
            (join[f'{col} lat'] != join['Lat']), f'{col} lat'] = join['Lat']
    
    join.loc[(join['Locations'].notnull()) & (join[col].notnull()) &
            (join[f'{col} long'] != join['Long']), f'{col} long'] = join['Long']
    
    return join.loc[:, :f'{col} long']
    
            
    

In [10]:
intv = update_locations(df, 'Interview', loc)
mf = update_locations(df, 'Migrated From', loc)
mt = update_locations(df, 'Migrated To', loc)
cr = update_locations(df, 'Current Residence', loc)

In [11]:
intv

,Interview,Interview lat,Interview long
0,"Denver, CO",39.730000,-104.990000
1,"Newark, NJ",40.140000,-74.670000
2,Washington DC,38.890000,-77.030000
3,"Seattle, WA",47.600000,-122.320000
4,"San Francisco, CA",37.730000,-122.460000
5,"New York City, NY",42.930000,-75.610000
6,"Santa Fe, NM",35.690000,-105.930000
7,"Dallas, TX",32.770000,-96.790000
8,"Jackson, WY",43.470000,-110.760000
9,"Portland, OR",45.510000,-122.670000


In [12]:
mf

,Migrated From,Migrated From lat,Migrated From long
0,"Los Angeles, CA",34.05,-118.24
1,"Dallas, TX",32.77,-96.79
2,Washington DC,38.89,-77.03
3,"Seattle, WA",47.60,-122.32
4,Washington DC,38.89,-77.03
5,"Newark, NJ",40.14,-74.67
6,"Denver, CO",39.73,-104.99
7,"Jackson, WY",43.47,-110.76
8,"Portland, OR",45.51,-122.67
9,"New York City, NY",42.93,-75.61


In [13]:
mt

,Migrated To,Migrated To lat,Migrated To long
0,"Dallas, TX",32.77,-96.79
1,"Jackson, WY",43.47,-110.76
2,"Portland, OR",45.51,-122.67
3,"Los Angeles, CA",34.05,-118.24
4,"Newark, NJ",40.14,-74.67
5,"Denver, CO",39.73,-104.99
6,"Denver, CO",39.73,-104.99
7,"Denver, CO",39.73,-104.99
8,"Denver, CO",39.73,-104.99
9,"San Francisco, CA",37.73,-122.46


In [14]:
cr

,Current Residence,Current Residence lat,Current Residence long
0,"Dallas, TX",32.77,-96.79
1,"Jackson, WY",43.47,-110.76
2,"Portland, OR",45.51,-122.67
3,"Los Angeles, CA",34.05,-118.24
4,"Newark, NJ",40.14,-74.67
5,"Denver, CO",39.73,-104.99
6,"Denver, CO",39.73,-104.99
7,"Denver, CO",39.73,-104.99
8,"Denver, CO",39.73,-104.99
9,"San Francisco, CA",37.73,-122.46


The final step is to take each subset that now has been updated and concatenate them together. This results in the updated Master Checklist. 

In [15]:
new_df = pd.concat([intv, mf, mt, cr], axis=1)
new_df

,Interview,Interview lat,Interview long,Migrated From,Migrated From lat,Migrated From long,Migrated To,Migrated To lat,Migrated To long,Current Residence,Current Residence lat,Current Residence long
0,"Denver, CO",39.730000,-104.990000,"Los Angeles, CA",34.05,-118.24,"Dallas, TX",32.77,-96.79,"Dallas, TX",32.77,-96.79
1,"Newark, NJ",40.140000,-74.670000,"Dallas, TX",32.77,-96.79,"Jackson, WY",43.47,-110.76,"Jackson, WY",43.47,-110.76
2,Washington DC,38.890000,-77.030000,Washington DC,38.89,-77.03,"Portland, OR",45.51,-122.67,"Portland, OR",45.51,-122.67
3,"Seattle, WA",47.600000,-122.320000,"Seattle, WA",47.60,-122.32,"Los Angeles, CA",34.05,-118.24,"Los Angeles, CA",34.05,-118.24
4,"San Francisco, CA",37.730000,-122.460000,Washington DC,38.89,-77.03,"Newark, NJ",40.14,-74.67,"Newark, NJ",40.14,-74.67
5,"New York City, NY",42.930000,-75.610000,"Newark, NJ",40.14,-74.67,"Denver, CO",39.73,-104.99,"Denver, CO",39.73,-104.99
6,"Santa Fe, NM",35.690000,-105.930000,"Denver, CO",39.73,-104.99,"Denver, CO",39.73,-104.99,"Denver, CO",39.73,-104.99
7,"Dallas, TX",32.770000,-96.790000,"Jackson, WY",43.47,-110.76,"Denver, CO",39.73,-104.99,"Denver, CO",39.73,-104.99
8,"Jackson, WY",43.470000,-110.760000,"Portland, OR",45.51,-122.67,"Denver, CO",39.73,-104.99,"Denver, CO",39.73,-104.99
9,"Portland, OR",45.510000,-122.670000,"New York City, NY",42.93,-75.61,"San Francisco, CA",37.73,-122.46,"San Francisco, CA",37.73,-122.46
